In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, SimpleRNN,Input
from tensorflow.keras.optimizers import Adam


In [3]:
# Load the dataset
df = pd.read_csv('../../dataset/preliminary_dataset.csv')  

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(80000, 1, 6) (80000,) (20000, 1, 6) (20000,)


# Model

In [5]:
def build_deep_rnn_bi_lstm():
    model = Sequential()

    # Input Layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    model.add(Bidirectional(LSTM(units=100, activation='relu', return_sequences=True)))
    model.add(Bidirectional(LSTM(units=100)))

    # Output Layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the Model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])


    return model


In [6]:

# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
import time
batch_size = X_train.shape[0]  # Use full dataset per batch to match experimental setup
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    # batch_size=batch_size,
    steps_per_epoch=1,
    verbose=1
)
end_time = time.time()



Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 0.6930 - mae: 0.4999 - mse: 0.2499
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 0.6895 - mae: 0.4982 - mse: 0.2482
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6860 - mae: 0.4964 - mse: 0.2464
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6822 - mae: 0.4945 - mse: 0.2445
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6780 - mae: 0.4923 - mse: 0.2424
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6732 - mae: 0.4899 - mse: 0.2400
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6677 - mae: 0.4870 - mse: 0.2373
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6614 - mae: 0.4837 - mse: 0.2341
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6541 - mae: 0.4798 - mse: 0.2305
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6456 - mae: 0.4753 - mse: 0.2263
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6359 - mae: 0.4700 - mse: 0.2215
Epoc

In [7]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 50 # e.g., 50
steps_per_epoch = 1# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch

# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
#micro seconds
time_per_step_us = time_per_step_ms * 1000

# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 50
Time per Step: 3.432718 sec (3432.72 ms)
Time per Step: 3432717.69 µs
Time per Epoch: 3.43 sec


In [8]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.16f}')


625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
Mean Squared Error for y_pred (MSE): 0.0000647377892165
Root Mean Squared Error (RMSE): 0.0080459796927690
Accuracy: 1.0000000000000000
